# Working with xAI Grok in Pixeltable

Pixeltable's xAI integration enables you to use Grok models for chat, reasoning, and image generation via the native xAI SDK.

### Prerequisites
- An xAI account with an API key (https://console.x.ai/)

### Important notes

- xAI API usage incurs costs based on your xAI plan
- The native xAI SDK (`xai-sdk`) provides access to the latest features including reasoning models
- Be mindful of sensitive data and consider security measures when integrating with external services


In [ ]:
%pip install -qU pixeltable xai-sdk

In [25]:
# For local development, use the local pixeltable path
import sys
sys.path.insert(0, '/Users/pierre/pixeltable')


In [26]:
import os
import getpass

if 'XAI_API_KEY' not in os.environ:
    os.environ['XAI_API_KEY'] = getpass.getpass('Enter your xAI API key: ')


Now let's create a Pixeltable directory to hold the tables for our demo.


In [27]:
import pixeltable as pxt

# Remove the 'xai_demo' directory and its contents, if it exists
pxt.drop_dir('xai_demo', force=True)
pxt.create_dir('xai_demo')


Created directory 'xai_demo'.


## Chat with Grok

Grok is xAI's flagship language model, designed to provide truthful, insightful answers. The `xai.chat` UDF uses the native xAI SDK which supports the latest features including the Responses API.

In [28]:
from pixeltable.functions import xai

# Create a table for chat conversations
chat_t = pxt.create_table('xai_demo.chat', {'prompt': pxt.String})

# Create message format for Grok
messages = [
    {'role': 'system', 'content': 'You are Grok, a helpful AI assistant created by xAI.'},
    {'role': 'user', 'content': chat_t.prompt}
]

# Add computed column using the native xAI chat UDF
chat_t.add_computed_column(
    response=xai.chat(messages=messages, model='grok-4')
)

# Extract just the response text
chat_t.add_computed_column(
    answer=chat_t.response['content']
)


Created table 'chat'.
Added 0 column values with 0 errors.
Added 0 column values with 0 errors.


No rows affected.

In [29]:
# Insert some prompts
chat_t.insert([
    {'prompt': 'What is the meaning of life, the universe, and everything?'},
    {'prompt': 'Explain quantum computing in simple terms.'},
    {'prompt': 'What makes a good cup of coffee?'},
])

Inserting rows into `chat`: 3 rows [00:00, 401.98 rows/s]
Inserted 3 rows with 0 errors.


3 rows inserted, 9 values computed.

In [30]:
# View the results
chat_t.select(chat_t.prompt, chat_t.answer).collect()


prompt,answer
"What is the meaning of life, the universe, and everything?","Ah, the ultimate question! According to Douglas Adams' *The Hitchhiker's Guide to the Galaxy*, the meaning of life, the universe, and everything is... 42. Of course, that's a cheeky, fictional supercomputer answer (Deep Thought spent 7.5 million years computing it, only to reveal that we don't really understand the question). In reality, philosophers, scientists, and thinkers have pondered this for millennia without a single, definitive answer. Some perspectives include: - **Existentialism ...... and perhaps ensure the survival of our species (as xAI might appreciate, we're driven by curiosity and the quest for knowledge). - **Religion/Spirituality**: Many traditions say it's about connection—to a higher power, to others, or to the world—through love, purpose, or enlightenment. - **Humor**: Maybe it's just to laugh at the absurdity of it all and enjoy the ride. If you're asking seriously, I'd say: Figure out what makes *you* tick, chase that, and don't panic. What's your take on it?"
Explain quantum computing in simple terms.,"Sure, I'll break down quantum computing in simple terms, like explaining it to a friend over coffee. I'll avoid too much jargon and keep it straightforward. ### What is Quantum Computing? At its core, quantum computing is a new way of doing calculations using the weird rules of quantum physics (the science of tiny particles like atoms and electrons). Traditional computers (like your laptop or phone) use ""bits"" as their building blocks—each bit is either a 0 or a 1, like a light switch that' ...... tasks like browsing the web or playing games are still best on regular computers. ### The Catch We're not there yet. Quantum computers today are small (with maybe 100-1,000 qubits) and make mistakes, so they're mostly for research. Experts think it'll take years (or decades) to make them practical and scalable. It's exciting, though—it's like the early days of classical computers in the 1940s. If you have a specific part you'd like me to expand on, like qubits or applications, just ask! 😊"
What makes a good cup of coffee?,"Ah, the quest for the perfect cup of coffee—a pursuit as old as time (or at least as old as the 15th century). As Grok, built by xAI, I'm no barista, but I've got the data to brew up some solid advice. What makes a good cup of coffee boils down (pun intended) to a few key factors: quality ingredients, proper technique, and a dash of personal preference. Let's break it down step by step. ### 1. **Start with Great Beans** - **Freshness is key**: Use freshly roasted beans. Coffee starts los ...... counts too—stale milk ruins everything. - Avoid over-extraction or under-brewing; taste and tweak. And hey, the mug matters—ceramic holds heat well. Ultimately, ""good"" is subjective. What I might call a stellar pour-over could be someone else's ""too weak"" tragedy. Experiment, track what you like, and remember: coffee is as much ritual as beverage. If you're chasing perfection, join a coffee subscription or hit up a local roaster. Got a specific brewing style in mind? I can dive deeper! ☕"


## Reasoning Models

The `grok-3-mini` model supports extended reasoning with the `reasoning_effort` parameter. This allows the model to "think" longer for complex problems.


In [31]:
# Create a table for reasoning tasks
reasoning_t = pxt.create_table('xai_demo.reasoning', {'problem': pxt.String})

# Create message format
messages = [
    {'role': 'system', 'content': 'You are a math expert. Show your work step by step.'},
    {'role': 'user', 'content': reasoning_t.problem}
]

# Use reasoning model with high effort for complex problems
reasoning_t.add_computed_column(
    response=xai.chat(
        messages=messages,
        model='grok-3-mini',
        reasoning_effort='high'
    )
)

# Extract answer and reasoning tokens used
reasoning_t.add_computed_column(answer=reasoning_t.response['content'])
reasoning_t.add_computed_column(reasoning_tokens=reasoning_t.response['usage']['reasoning_tokens'])


Created table 'reasoning'.
Added 0 column values with 0 errors.
Added 0 column values with 0 errors.
Added 0 column values with 0 errors.


No rows affected.

In [32]:
# Insert math problems
reasoning_t.insert([
    {'problem': 'What is 101 * 3?'},
    {'problem': 'If a train travels 120 miles in 2 hours, what is its average speed?'},
])


Inserting rows into `reasoning`: 2 rows [00:00, 807.37 rows/s]
Inserted 2 rows with 0 errors.


2 rows inserted, 8 values computed.

In [33]:
# View the results with reasoning tokens
reasoning_t.select(reasoning_t.problem, reasoning_t.answer, reasoning_t.reasoning_tokens).collect()


problem,answer,reasoning_tokens
"If a train travels 120 miles in 2 hours, what is its average speed?","To find the average speed of the train, use the formula for average speed: \[ \text{average speed} = \frac{\text{total distance}}{\text{total time}} \] Step 1: Identify the total distance and total time from the given information. - Total distance = 120 miles - Total time = 2 hours Step 2: Substitute the values into the formula: \[ \text{average speed} = \frac{120 \text{ miles}}{2 \text{ hours}} \] Step 3: Perform the division: \[ 120 \div 2 = 60 \] Step 4: Include the units. The average speed is 60 miles per hour. \[ \boxed{60} \]",311
What is 101 * 3?,"To compute \(101 \times 3\), I will show the steps clearly. Step 1: Break down the number 101 into simpler parts. We can write 101 as \(100 + 1\). Step 2: Apply the distributive property of multiplication: \[ (100 + 1) \times 3 = (100 \times 3) + (1 \times 3) \] Step 3: Perform the multiplications: \[ 100 \times 3 = 300 \] \[ 1 \times 3 = 3 \] Step 4: Add the results: \[ 300 + 3 = 303 \] Alternatively, using the standard multiplication algorithm: - Multiply 3 by the units digit of 101 (which is 1): \(3 \times 1 = 3\). - Multiply 3 by the tens digit of 101 (which is 0): \(3 \times 0 = 0\). - Multiply 3 by the hundreds digit of 101 (which is 1): \(3 \times 1 = 3\). - Combine the results: 303. Both methods confirm the result. \[ \boxed{303} \]",731


## Vision / Image Understanding

Grok can analyze images and answer questions about them using the `xai.vision` UDF. This uses the native xAI SDK with vision-capable models like `grok-4`.


In [ ]:
# Create a table for image analysis
vision_t = pxt.create_table('xai_demo.vision', {'image_url': pxt.String, 'question': pxt.String})

# Add computed column for vision analysis
vision_t.add_computed_column(
    analysis=xai.vision(
        prompt=vision_t.question,
        image_url=vision_t.image_url,
        model='grok-4',
        detail='high'
    )
)

# Extract the description
vision_t.add_computed_column(answer=vision_t.analysis['content'])

Created table 'vision'.
Added 0 column values with 0 errors.
Added 0 column values with 0 errors.


No rows affected.

In [35]:
# Insert images for analysis
vision_t.insert([
    {
        'image_url': 'https://science.nasa.gov/wp-content/uploads/2023/09/web-first-images-release.png',
        'question': 'What is shown in this image? Describe it in detail.'
    },
    {
        'image_url': 'https://upload.wikimedia.org/wikipedia/commons/thumb/1/15/Cat_August_2010-4.jpg/1200px-Cat_August_2010-4.jpg',
        'question': 'What animal is in this image and what is it doing?'
    },
])

ExprEvalError: Expression evaluation failed with an error of type `builtins.TypeError`:
chat(messages=[{'role': 'user', 'content': [{'type': 'text', 'text': question}, {'type': 'image_url', 'image_url': {'url': image_url, 'detail': 'high'}}]}], model='grok-4')

In [ ]:
# View the image analysis results
vision_t.select(vision_t.question, vision_t.answer).collect()

## Image Generation

xAI offers image generation capabilities with the `grok-2-image` model using the native xAI SDK.


In [ ]:
# Create a table for image generation
img_t = pxt.create_table('xai_demo.images', {'prompt': pxt.String})

# Add computed column for image generation
img_t.add_computed_column(
    generated_image=xai.image_generations(img_t.prompt, model='grok-2-image')
)


In [ ]:
# Insert image prompts
img_t.insert([
    {'prompt': 'A futuristic cityscape at sunset with flying cars'},
    {'prompt': 'A cute robot helping in a garden'},
])

In [ ]:
# View the generated images
img_t.select(img_t.prompt, img_t.generated_image).collect()

## Related Cookbooks

Explore these Pixeltable cookbooks to build more advanced applications with Grok:

- [LLM Tool Calling](../cookbooks/agents/llm-tool-calling.ipynb) - Build agents with function calling
- [Agent Memory Patterns](../cookbooks/agents/pattern-agent-memory.ipynb) - Implement persistent memory for conversational agents
- [RAG Pipeline Patterns](../cookbooks/agents/pattern-rag-pipeline.ipynb) - Build retrieval-augmented generation systems
- [Data Lookup Patterns](../cookbooks/agents/pattern-data-lookup.ipynb) - Connect LLMs to your data

## Learn More

**xAI Resources:**
- [xAI Documentation](https://docs.x.ai/)
- [xAI Console](https://console.x.ai/)
- [Grok Models](https://docs.x.ai/docs/models)
- [Responses API Guide](https://docs.x.ai/docs/guides/responses-api)
- [Reasoning Models Guide](https://docs.x.ai/docs/guides/reasoning)
